In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import scipy
import requests 
import os 
import datetime
from scipy import stats
import netCDF4 as nc
from mpl_toolkits.basemap import Basemap

In [ ]:
stn_coord=pd.read_csv('../data/observation/coords.csv')
stn=stn_coord['stn']
stn_lat=stn_coord['lat']
stn_lon=stn_coord['lon']

In [ ]:
# ploting study area and stations location

fig=plt.figure(figsize=(8, 8))
m=Basemap(projection='cyl', resolution='h',llcrnrlat=3, urcrnrlat=15, llcrnrlon=2, urcrnrlon=15)
m.etopo(scale=5)
m.drawcoastlines(linewidth=1, color='black', zorder=2)
m.drawcountries(linewidth=1, color='black', zorder=3)

m.scatter(stn_lon, stn_lat, latlon=True, c='red', s=50, edgecolors='black', zorder=3, cmap='jet')

for i in range(len(stn_lat)):
    plt.annotate(stn[i], (stn_lon[i], stn_lat[i]+0.2))
# Map (long, lat) to (x, y) for plotting
#  x, y = m(-122.3, 47.6)
#  plt.plot(x, y, 'ok', markersize=5)
#  plt.text(x, y, ' Seattle', fontsize=12);
plt.xticks(range(2, 16, 2))
plt.yticks(range(3, 16, 2))

plt.savefig('../plots/study.png', dpi=150)

In [ ]:
# Daily visibility data for 47 stations in Nigeria from Dec 2021 to Mar 2022
vis=pd.read_csv('../data/observation/vis_12_2021_03_2022.csv', index_col=[0])
vis.index=pd.to_datetime(vis.index, format='%d/%m/%Y')
vis

In [ ]:
# cap visibility maximum to 10km
# training set from Dec 2021 to Jan 2022, possible to add Feb?, validation for Mar??
vis.columns
for i in vis.columns:
    vis[i]=np.where(vis[i]>10, 10, vis[i])
vis_train=vis.iloc[:90]
vis_test=vis.iloc[90:]
vis

In [ ]:
vis_train['KAN'].describe()
vis['KAN'].hist(bins=5)

In [ ]:
missing_train=np.isnan(vis_train).sum()
missing_test=np.isnan(vis_test).sum()
# print(missing_test)
# print(missing_train)

name=missing_train.index
print(name)
print(len(name))

In [ ]:
# plotting missing values for each station, the red line shows position of 20% missing values
bar_width=0.25
fig, (ax1, ax2) =plt.subplots(2, figsize=(10,8))

ax1.bar(name, missing_train)
ax1.set_title('(a)')
ax1.tick_params(axis='x', labelrotation=90)
ax1.set_ylim([0,90])
ax1.axhline(y=(len(vis_train)/100)*20, color='r', linestyle='--')
ax1.set_ylabel('Days (Dec 2021 to Feb 2022)')

ax2.bar(name, missing_test)
ax2.set_title('(b)')
ax2.tick_params(axis='x', labelrotation=90)
ax2.set_ylim([0,30])
ax2.axhline(y=(len(vis_test)/100)*20, color='r', linestyle='--')
ax2.set_ylabel('Days (Mar 2022)')

fig.tight_layout(pad=1)

fig.savefig('../plots/missing_data.png', dpi=150)

In [ ]:
# bsc=xr.open_dataset('../data/models/ALADIN_DUST/2021120100_3H_ALADIN_DUST.nc')
# bsc

In [ ]:
# all_models=['ALADIN_DUST', 
# all_variables=['od550_dust'
all_day1_lead=[('a', 'ALADIN_DUST', 'od550_dust', (11, 19)),
               ('b', 'BSC_DREAM8b_V2', 'OD550_DUST', (7, 15)),
               ('c', 'MACC-ECMWF', 'OD550_DUST', (11, 19)),
               ('d', 'DREAM8-MACC', 'OD550_DUST', (11, 19)),
               ('e', 'ICON-ART', 'OD550_DUST', (11, 19)),
               ('f', 'LOTOSEUROS', 'OD550_DUST', (11, 19)),
               ('g', 'MEDIAN', 'OD550_DUST', (7, 15)),
               ('h', 'NASA-GEOS', 'OD550_DUST', (11, 19)),
               ('i', 'NCEP-NGAC', 'OD550_DUST', (11, 19)),
               ('j', 'NMMB-BSC', 'od550_dust', (7, 15)),
               ('k', 'NOA-WRF-CHEM', 'OD550_DUST', (7, 15)),
               ('l', 'SILAM', 'OD550_DUST', (11, 19))]
               

In [ ]:
# for h, i, j, k in all_day1_lead:
#     print(k[1])

In [ ]:
#11 19
# bsc.time.to_series()[19]
h='a'
i='ALADIN_DUST'
j='od550_dust'
k=(11, 19)
k[0]
model_dir=f'../data/models/{i}/'
nc_files=os.listdir(model_dir)
len(nc_files)

In [ ]:
# for h, i, j, k in all_day1_lead:
#     # print(k[1])
#     model_dir=f'../data/models/{i}/'
#     nc_files=os.listdir(model_dir)
    # print(nc_files)
    
    
    

In [ ]:
day1_lead_all=pd.DataFrame(columns=['stn', 'lon', 'lat'])

for nc_file in nc_files:
    bsc=xr.open_dataset(model_dir+nc_file)
    # print(bsc)
    day1_lead=[['stn', 'lon', 'lat', bsc[j].time.to_series()[k[0]]]]
    for i in range(len(stn)):
        day1_bsc=bsc[j].isel(time=k[0]).sel(lat=stn_lat[i], lon=stn_lon[i], method='nearest')
        day1_lead.append([stn[i], stn_lon[i], stn_lat[i], day1_bsc.to_dict()['data']])
    day1_lead=pd.DataFrame(day1_lead[1:len(day1_lead)], columns=day1_lead[0])
    day1_lead_all=day1_lead_all.merge(day1_lead, left_on=['stn', 'lon', 'lat'], right_on=['stn', 'lon', 'lat'], how='outer')
#data for 1 day leadtime for all sations
day1_lead_all

In [ ]:
day1_lead_all.index=day1_lead_all['stn']
day1_lead_all_t=day1_lead_all.transpose()
day1_lead_all_t=day1_lead_all_t.drop(['lon', 'lat', 'stn'])

# multiply AOD by 10
# day1_lead_all_t = day1_lead_all_t * 10

# day1_lead_all_t = day1_lead_all_t.reset_index()
day1_lead_all_t

In [ ]:
day1_lead_all_t.index=pd.to_datetime(day1_lead_all_t.index, format="%Y-%m-%d").date
day1_lead_all_t.index=pd.to_datetime(day1_lead_all_t.index)
day1_lead_all_t

In [ ]:
# na.array(vis['YOL'])
# merged = pd.merge(vis[["YOL"]], day1_lead_all_t[["YOL"]], left_index=True, right_on="index")
# merged = pd.concat([vis[["YOL"]], day1_lead_all_t[["YOL"]]], axis=1)
# merged.columns = ["Visibility", "AOD"]
# merged

In [ ]:
merged = pd.merge(vis['ABE'], day1_lead_all_t['ABE'], how='inner', left_index=True, right_index=True)
merged.columns = ["Visibility", "AOD"]
merged.dropna(inplace=True)
merged

# Regression

In [ ]:
# plt.rc('font', family='serif')
# plt.rc('xtick', labelsize='x-small')
# plt.rc('ytick', labelsize='x-small')
cor_ls = []

fig = plt.figure(figsize=(12,16))
# fig.subplots_adjust(hspace=0.1, wspace=0.1)

for i, j in enumerate(stn):
    merged = pd.concat([vis[[j]], day1_lead_all_t[[j]]], axis=1)
    merged.columns = ["Visibility", "AOD"]
    
    a=merged.dropna()
#     regression part
    slope, intercept, r_value, p_value, std_err = stats.linregress(list(a['AOD']),list(a['Visibility']))
    line = (slope*a['AOD'])+intercept
    
    # append correlation
    cor_ls.append([stn_lat[i], stn_lon[i], j, round(r_value, 2), round(p_value, 2)])
    
    ax = fig.add_subplot(8,6,i+1, )
    ax.scatter(merged['AOD'], merged['Visibility'], color = 'k')
    ax.plot(list(a['AOD']), line, 'r')
    title = j+': y={:.2f}x+{:.2f} \n '.format(slope,intercept)  + 'r={:.2f}, '.format(r_value) + 'p-value={:.2f}'.format(p_value)

    ax.set_title(title, fontsize = 10)
#     ax.set_xlim([0, 1.5])
    ax.set_ylim([-1, 11])
    
#     ax.set_ylabel('Visibility (in Km)')
#     ax.set_xlabel('bbb')
# fig.suptitle('a')
fig.supxlabel('AOD')
fig.supylabel('Visibility (in Km)')
fig.tight_layout(rect=(0.0,0,1,1))
    
# plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
# plt.xlabel("common X")
# plt.ylabel("common Y")    
#     plt.subplot(7,7,i+1)
#     plt.scatter(merged['AOD'], merged['Visibility'], color = 'k', ls='solid')
#     plt.title(j)
# plt.xlabel('AOD')
# plt.ylabel('Visibility (in Km)')
#     plt.text(j)
#     for j in stn:
        
#         plt.subplot(2, 3, i)
#         plt.scatter(merged['AOD'], merged['Visibility'])
# # #         plt.text(0.5, 0.5, str((2, 3, i)),
# # #                  fontsize=18, ha='center')
            


# # fig = plt.figure(figsize=(3,3))
# fig, axs = plt.subplots(nrows=7, ncols=7, figsize=(15,15))
# plt.subplots_adjust(hspace=0.2)
# for j, ax in zip(stn, axs.ravel()):
#     print(j, ax)
#     merged = pd.concat([vis[[j]], day1_lead_all_t[[j]]], axis=1)
#     merged.columns = ["Visibility", "AOD"]
# #     merged['AOD'].scatter(merged['Visibility'],ax=ax)
#     plt.scatter(merged['AOD'], merged['Visibility'],ax=ax)
# #     plt.figure()
# #     plt.scatter(merged['AOD'], merged['Visibility'],color='k', ls='solid')
# #     plt.show()
    
    

In [ ]:
cor_df = pd.DataFrame(cor_ls, columns=["Lat", "Long", "Station", "r_value", "p_value"])
cor_df['p'] = 2
cor_df.loc[cor_df['p_value'] > 0.05, 'p'] = 1
cor_df

In [ ]:
# cor_df.to_csv("cor_files/day1/ALADIN_1d.csv", index=False)
cor_df.to_csv("cor_files/day2/ALADIN_2d.csv", index=False)
# cor_df.to_csv("cor_files/day3/ALADIN_3d.csv", index=False)

In [ ]:
# plot coastlines with Basemap
plt.figure(figsize=(8, 8))
m = Basemap(projection='cyl', resolution='h',
llcrnrlat=3, urcrnrlat=15,
llcrnrlon=2, urcrnrlon=15)
# m.drawmapboundary(fill_color='#DDEEFF')
# m.fillcontinents(color='#FFEEDD')
m.drawcoastlines(color='grey', zorder=2)
m.drawcountries(color='black', zorder=2)
# m.shadedrelief()
m.drawstates()

# m.readshapefile('./nigeria_states/nigeria_states', 'nigeria_states')

# plot locations
m.scatter(cor_df["Long"], cor_df["Lat"], latlon=True, 
          c=cor_df["r_value"], s=cor_df["p"]*100,
          edgecolors="black", zorder=3,
          cmap='jet', alpha=0.7)

# create colorbar and legend
m.colorbar(label=r'r', ticks=np.linspace(1, -1, 9), shrink=0.755)
# plt.set_ticks(np.linspace(1, -1, 9))
plt.xticks(range(2, 16, 2))
plt.yticks(range(3, 16, 2))
plt.show()

### Probablity of exceedence based on ensemble 

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# simulating some data

tseries=pd.Series(np.random.random(100))
tseries.tail()

95    0.323252
96    0.121945
97    0.886595
98    0.464036
99    0.005237
dtype: float64

In [3]:
# using 3 lags -- the no. of observation we look back to model the next value of the series
N_LAGS=3

# constructing a set of observations based on past recent values (lags)

tseries_df=pd.concat([tseries.shift(i) for i in range(N_LAGS, -1, -1)], axis=1)

# assigning column names and dropping na's

tseries_df.columns=['t-'+str(i) for i in list(reversed(range(N_LAGS)))]+['t+1']
tseries_df=tseries_df.dropna().reset_index(drop=True)
tseries_df

,t-2,t-1,t-0,t+1
0,0.615602,0.939098,0.615571,0.155945
1,0.939098,0.615571,0.155945,0.369595
2,0.615571,0.155945,0.369595,0.882565
3,0.155945,0.369595,0.882565,0.826987
4,0.369595,0.882565,0.826987,0.078323
...,...,...,...,...
92,0.007439,0.853912,0.165510,0.323252
93,0.853912,0.165510,0.323252,0.121945
94,0.165510,0.323252,0.121945,0.886595
95,0.323252,0.121945,0.886595,0.464036


In [45]:
# the target variable is the column 't+1' representing the next value of the series
y=tseries_df['t+1']

# removing the target to create predictor variables
X=tseries_df.drop('t+1', axis=1)

# train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

# setting threshold to the 95th percentile
thr=np.quantile(y_train, .95)


In [46]:
# transforming the numeric target to binary one
y_train_binary=(y_train >= thr).astype(int)
y_test_binary=(y_test >= thr).astype(int)

# training a probabilistic classifier
classifier=LogisticRegression()
classifier.fit(X_train, y_train_binary)

# predicting exceedance probability
exceedance_prob_clf=classifier.predict_proba(X_test)[:,1]
exceedance_prob_clf

array([0.07956216, 0.04532982, 0.06864187, 0.05772812, 0.0542389 ,
       0.05285436, 0.07259731, 0.04596259, 0.07511233, 0.06575675,
       0.05140069, 0.06936529, 0.06163599, 0.04639801, 0.08802188,
       0.05574677, 0.07020001, 0.07230755, 0.07465677, 0.05124361,
       0.06285898, 0.06583928, 0.04887018, 0.07053539, 0.04677576,
       0.06125991, 0.0436467 , 0.05236644, 0.06141769, 0.05191043,
       0.06158747, 0.05908756, 0.06335157])

In [15]:
sum(y_test_binary)

0

### Ensemble approach

In [19]:
tseries=pd.Series(np.random.random(100))
# using 3 lags -- the no. of observation we look back to model the next value of the series
N_LAGS=3

# constructing a set of observations based on past recent values (lags)

tseries_df=pd.concat([tseries.shift(i) for i in range(N_LAGS, -1, -1)], axis=1)

# assigning column names and dropping na's

tseries_df.columns=['t-'+str(i) for i in list(reversed(range(N_LAGS)))]+['t+1']
tseries_df=tseries_df.dropna().reset_index(drop=True)

# the target variable is the column 't+1' representing the next value of the series
y=tseries_df['t+1']

# removing the target to create predictor variables
X=tseries_df.drop('t+1', axis=1)

# train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

In [41]:
# using the RF as ensemble of trees

from sklearn.ensemble import RandomForestRegressor

# training the model
forecasting_ensemble=RandomForestRegressor()

# here we use numerical target
# which represent the next value of the time series
forecasting_ensemble.fit(X_train, y_train)

# retrieving the predictions from the trees
per_tree_pred = [tree.predict(X_test) for tree in forecasting_ensemble.estimators_]
per_tree_df = pd.DataFrame(per_tree_pred).T

# computing the ratio of trees which predict a value above the threshold
exceedance_prob_fe = per_tree_df.apply(lambda x: np.mean(x > thr), axis=1).values
exceedance_prob_fe

array([0.18, 0.02, 0.17, 0.02, 0.07, 0.  , 0.  , 0.04, 0.4 , 0.01, 0.11,
       0.2 , 0.  , 0.  , 0.04, 0.04, 0.01, 0.24, 0.02, 0.01, 0.  , 0.  ,
       0.  , 0.29, 0.  , 0.1 , 0.01, 0.  , 0.  , 0.  , 0.23, 0.17, 0.  ])

In [47]:
from sklearn.metrics import brier_score_loss, roc_auc_score

roc_auc_score(y_true=y_test_binary, y_score=exceedance_prob_fe)
roc_auc_score(y_true=y_test_binary, y_score=exceedance_prob_clf)

brier_score_loss(y_true=y_test_binary, y_prob=exceedance_prob_fe)
brier_score_loss(y_true=y_test_binary, y_prob=exceedance_prob_clf)

0.029850353287022707

In [49]:
np.mean([True,True,False])


0.6666666666666666